# Results

## Table of Contents
* [1)*American Put Prices* Applying Least-Square Monte Carlo with Longstaff-Schwartz Parameters](#first-bullet)
    - [1a) Black-Scholes (*Akin to Table 1 in Longstaff-Schwartz*)](#firsta-bullet)
    - [1b) Jump Merton](#firstb-bullet)
    - [1c) Constant Elasticity of Variance (CEV)](#firstc-bullet)
* [2) *American Put Prices* Applying Least-Square Monte Carlo with Our Own Parameter Values](#second-bullet)
    - [2a) Black-Scholes](#seconda-bullet)
    - [2b) Jump Merton](#secondb-bullet)
    - [2c) Constant Elasticity of Variance (CEV)](#secondc-bullet)

## Setup

In [19]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import os

# local
from DC import DynamicChebyshev
from LSM import MonteCarloOptionPricing
from FD import AmericanPutFiniteDifference

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1) *American Put Prices* Applying Least-Square Monte Carlo with Longstaff-Schwartz Parameters <a class="anchor" id="first-bullet"></a>
The simulation is based on 100,000 (50,000 plus 50,000 antithetic) paths for the stock price.

In [20]:
# Parameter inputs
r = 0.06  # risk-free rate
K = 40.   # strike price
λ = 0.4 # intensity rate of the Poisson process
dim = 10_000  # number of simulation paths
n = 50  # number of exercise rights per year
seed = 15001  # random seed for reproducibility
n_chebyshev_pol = 100
use_AV = True  # antithetic variates
poly_degree = 3  # polynomial degree for LSM regression

In [21]:
# Varying option parameters
S0_values = [36, 38, 40, 42, 44]
σ_values = [0.20, 0.40]
T_values = [1, 2]

### 1a) Black-Scholes (*Akin to Table 1 in Longstaff-Schwartz*) <a class="anchor" id="firsta-bullet"></a>


In [22]:
# Store results
Option_Pricing_Results_BS = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_BS = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Calculate European Put using Black-Scholes
            BS_Price = MC_Option_Pricing_BS.BS_option_value('put')
            
            # 2) Compute American Put using LSM
            MC_Option_Pricing_BS.GeometricBrownianMotion_vec() # simulate GBM paths
            American_put_price = MC_Option_Pricing_BS.american_option_LSM(poly_degree, otype='put') # price option
            
            # 3) Calculate premium
            Option_Premium = American_put_price[0] - BS_Price 
            
            # 4) Compute American Put using Implicit Finite Difference
            Implicit_Finite_Difference = AmericanPutFiniteDifference(K, r, M = 1_000)
            FD = Implicit_Finite_Difference.implicit_FD(S0, σ, T, N=n)
            
            # 5) Compute American Put using Dynamic Chebyshev 
            DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, n, n_chebyshev_pol, seed, use_AV)
            
            # a) offline phase
            domain = DC_option_pricing.calculate_truncated_domain_GBM()
            xknots = DC_option_pricing.calculate_nodal_points(domain)
            BS = DC_option_pricing.generate_GBM_path(xknots)
            Γ = DC_option_pricing.compute_generalized_moments(domain, xknots)

            # b) online phase
            DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots, Γ)
            
            # 6) Store the results
            Option_Pricing_Results_BS.append((S0, σ, T, BS_Price, American_put_price[0], Option_Premium, FD, American_put_price[3], DC[0], DC[1]))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, EUR Put: {BS_Price:.4f}, LSM: {American_put_price[0]:.4f}, Premium: {Option_Premium:.4f}, F.D.: {FD:.4f}, s.e.: {American_put_price[3]:.4f}, D.C.: {DC[0]:.4f}, s.e.: {DC[1]:.4f}')

S0/Sigma/T: 36/0.2/1, EUR Put: 3.8443, LSM: 4.4760, Premium: 0.6317, F.D.: 4.4650, s.e.: 0.0291, D.C.: 4.5445, s.e.: 0.0772
S0/Sigma/T: 36/0.2/2, EUR Put: 3.7630, LSM: 4.8401, Premium: 1.0771, F.D.: 4.8114, s.e.: 0.0358, D.C.: 4.9248, s.e.: 0.0934
S0/Sigma/T: 36/0.4/1, EUR Put: 6.7114, LSM: 7.0991, Premium: 0.3877, F.D.: 7.0754, s.e.: 0.0614, D.C.: 7.2365, s.e.: 0.1195
S0/Sigma/T: 36/0.4/2, EUR Put: 7.7000, LSM: 8.4803, Premium: 0.7802, F.D.: 8.3893, s.e.: 0.0728, D.C.: 8.6651, s.e.: 0.1397
S0/Sigma/T: 38/0.2/1, EUR Put: 2.8519, LSM: 3.2598, Premium: 0.4079, F.D.: 3.2368, s.e.: 0.0309, D.C.: 3.3359, s.e.: 0.0723
S0/Sigma/T: 38/0.2/2, EUR Put: 2.9906, LSM: 3.7552, Premium: 0.7646, F.D.: 3.7185, s.e.: 0.0361, D.C.: 3.8472, s.e.: 0.0891
S0/Sigma/T: 38/0.4/1, EUR Put: 5.8343, LSM: 6.1525, Premium: 0.3182, F.D.: 6.1224, s.e.: 0.0601, D.C.: 6.2928, s.e.: 0.1168
S0/Sigma/T: 38/0.4/2, EUR Put: 6.9788, LSM: 7.6809, Premium: 0.7021, F.D.: 7.5687, s.e.: 0.0723, D.C.: 7.8383, s.e.: 0.1380
S0/Sigma

### 1b) Jump Merton <a class="anchor" id="firstb-bullet"></a>

In [23]:
λ = 0.4 # intensity rate of the Poisson process
α = -0.5 # Mean of log-normal jump size
β = 0.4 # Volatility of log-normal jump size

In [24]:
# Store results
Option_Pricing_Results_JM = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_JM = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Calculate European Put using Jump Merton semi-closed solution
            JumpMerton_price = MC_Option_Pricing_JM.merton_jump_option_value(α, β)
            
            # 2) Compute American Put using LSM
            MC_Option_Pricing_JM.MertonJumpDiffusion_vec(α, β) # simulate jump paths
            American_put_price = MC_Option_Pricing_JM.american_option_LSM(poly_degree, otype='put') # price option
            
            # 3) Calculate premium
            Option_Premium = American_put_price[0] - JumpMerton_price 
            
            # 4) Compute American Put using Dynamic Chebyshev 
            DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, n, n_chebyshev_pol, seed, use_AV)
            # a) offline phase
            domain = DC_option_pricing.calculate_truncated_domain_JumpMerton(α, β)
            xknots = DC_option_pricing.calculate_nodal_points(domain)
            JM = DC_option_pricing.generate_Jump_path(xknots, α, β)
            Γ = DC_option_pricing.compute_generalized_moments(domain,xknots)

            # b) online phase
            DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots,Γ)
            
            # 5) Store the results
            Option_Pricing_Results_JM.append((S0, σ, T, JumpMerton_price, American_put_price[0], Option_Premium, DC[0]))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, EUR Put: {JumpMerton_price:.4f}, LSM: {American_put_price[0]:.4f}, Premium: {Option_Premium:.4f}, D.C.: {DC[0]:.4f}')

S0/Sigma/T: 36/0.2/1, EUR Put: 5.7636, LSM: 6.2070, Premium: 0.4434, D.C.: 6.8556
S0/Sigma/T: 36/0.2/2, EUR Put: 7.1273, LSM: 8.1578, Premium: 1.0305, D.C.: 8.6021
S0/Sigma/T: 36/0.4/1, EUR Put: 8.0853, LSM: 8.5250, Premium: 0.4396, D.C.: 9.8797
S0/Sigma/T: 36/0.4/2, EUR Put: 9.8050, LSM: 10.7811, Premium: 0.9761, D.C.: 12.0403
S0/Sigma/T: 38/0.2/1, EUR Put: 5.0684, LSM: 5.4040, Premium: 0.3356, D.C.: 6.0276
S0/Sigma/T: 38/0.2/2, EUR Put: 6.5609, LSM: 7.5066, Premium: 0.9457, D.C.: 7.8989
S0/Sigma/T: 38/0.4/1, EUR Put: 7.3232, LSM: 7.7162, Premium: 0.3930, D.C.: 9.1396
S0/Sigma/T: 38/0.4/2, EUR Put: 9.1839, LSM: 10.0724, Premium: 0.8885, D.C.: 11.3532
S0/Sigma/T: 40/0.2/1, EUR Put: 4.5068, LSM: 4.7774, Premium: 0.2706, D.C.: 5.3177
S0/Sigma/T: 40/0.2/2, EUR Put: 6.0619, LSM: 6.9193, Premium: 0.8574, D.C.: 7.2888
S0/Sigma/T: 40/0.4/1, EUR Put: 6.6461, LSM: 7.0019, Premium: 0.3558, D.C.: 8.4338
S0/Sigma/T: 40/0.4/2, EUR Put: 8.6154, LSM: 9.4270, Premium: 0.8115, D.C.: 10.7300
S0/Sigma/T:

### 1c) Constant Elasticity of Variance (CEV) <a class="anchor" id="firstc-bullet"></a>

In [25]:
γ = 1.5 # parameter governing elasticity with respect to price, if set = 2 then we obtain BS prices

In [26]:
# Store results
Option_Pricing_Results_CEV = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_CEV = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Compute American Put using LSM
            MC_Option_Pricing_CEV.CEV(γ) # simulate CEV paths
            American_put_price = MC_Option_Pricing_CEV.american_option_LSM(poly_degree, otype='put') # price option
            
            # 2) Compute American Put using Dynamic Chebyshev
            DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, n, n_chebyshev_pol, seed, use_AV)
            # a) offline phase
            domain = DC_option_pricing.calculate_truncated_domain_GBM()
            xknots = DC_option_pricing.calculate_nodal_points(domain)
            CEV = DC_option_pricing.generate_CEV_path(xknots, γ)
            Γ = DC_option_pricing.compute_generalized_moments(domain, xknots)

            # b) online phase
            DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots, Γ)
            
            # 3) Store the results
            Option_Pricing_Results_CEV.append((S0, σ, T, American_put_price[0], DC[0]))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, LSM: {American_put_price[0]:.4f}, D.C.: {DC[0]:.4f}')

S0/Sigma/T: 36/0.2/1, LSM: 3.9521, D.C.: 4.0606
S0/Sigma/T: 36/0.2/2, LSM: 3.9042, D.C.: 4.1675
S0/Sigma/T: 36/0.4/1, LSM: 4.0818, D.C.: 5.7912
S0/Sigma/T: 36/0.4/2, LSM: 4.2452, D.C.: 6.6713
S0/Sigma/T: 38/0.2/1, LSM: 1.9563, D.C.: 2.6215
S0/Sigma/T: 38/0.2/2, LSM: 1.9376, D.C.: 2.8772
S0/Sigma/T: 38/0.4/1, LSM: 2.7075, D.C.: 4.7465
S0/Sigma/T: 38/0.4/2, LSM: 3.0111, D.C.: 5.7437
S0/Sigma/T: 40/0.2/1, LSM: 0.6246, D.C.: 1.6299
S0/Sigma/T: 40/0.2/2, LSM: 0.6965, D.C.: 1.9655
S0/Sigma/T: 40/0.4/1, LSM: 1.7271, D.C.: 3.8732
S0/Sigma/T: 40/0.4/2, LSM: 2.0944, D.C.: 4.9426
S0/Sigma/T: 42/0.2/1, LSM: 0.1642, D.C.: 0.9772
S0/Sigma/T: 42/0.2/2, LSM: 0.2336, D.C.: 1.3288
S0/Sigma/T: 42/0.4/1, LSM: 1.0499, D.C.: 3.1390
S0/Sigma/T: 42/0.4/2, LSM: 1.4388, D.C.: 4.2503
S0/Sigma/T: 44/0.2/1, LSM: 0.0359, D.C.: 0.5641
S0/Sigma/T: 44/0.2/2, LSM: 0.0729, D.C.: 0.8894
S0/Sigma/T: 44/0.4/1, LSM: 0.6016, D.C.: 2.5261
S0/Sigma/T: 44/0.4/2, LSM: 0.9627, D.C.: 3.6511


## 2) *American Put Prices* Applying Least-Square Monte Carlo with Our Own Parameter Values <a class="anchor" id="second-bullet"></a>
The simulation is based on 100,000 (50,000 plus 50,000 antithetic) paths for the stock price.

In [27]:
# Parameter inputs
r = 0.04  # risk-free rate
K = 100.   # strike price
λ = 0.4 # intensity rate of the Poisson process
dim = 1_000  # number of simulation paths
n = 50  # number of exercise rights per year
seed = 15001  # random seed for reproducibility
use_AV = True  # antithetic variates
poly_degree = 3  # polynomial degree for LSM regression

In [28]:
# Option parameters
S0_values = [90, 95, 100, 105, 110]
σ_values = [0.20, 0.40]
T_values = [1, 2]

## 2a) Black-Scholes <a class="anchor" id="seconda-bullet"></a>

In [29]:
# Store results
Results_BS = []

# Store results
Option_Pricing_Results_BS = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_BS = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Calculate European Put using Black-Scholes
            BS_Price = MC_Option_Pricing_BS.BS_option_value('put')
            
            # 2) Compute American Put using LSM
            MC_Option_Pricing_BS.GeometricBrownianMotion_vec() # simulate GBM paths
            American_put_price = MC_Option_Pricing_BS.american_option_LSM(poly_degree, otype='put') # price option
            
            # 3) Calculate premium
            Option_Premium = American_put_price[0] - BS_Price 
            
            # 4) Compute American Put using Implicit Finite Difference
            Implicit_Finite_Difference = AmericanPutFiniteDifference(K, r, M = 1_000)
            FD = Implicit_Finite_Difference.implicit_FD(S0, σ, T, N=n)
            
            # 5) Compute American Put using Dynamic Chebyshev 
            DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, n, n_chebyshev_pol, seed, use_AV)
            
            # a) offline phase
            domain = DC_option_pricing.calculate_truncated_domain_GBM()
            xknots = DC_option_pricing.calculate_nodal_points(domain)
            BS = DC_option_pricing.generate_GBM_path(xknots)
            Γ = DC_option_pricing.compute_generalized_moments(domain, xknots)

            # b) online phase
            DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots, Γ)
            
            # 6) Store the results
            Results_BS.append((S0, σ, T, BS_Price, American_put_price[0], Option_Premium, FD, American_put_price[3], DC[0], DC[1]))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, EUR Put: {BS_Price:.4f}, LSM: {American_put_price[0]:.4f}, Premium: {Option_Premium:.4f}, F.D.: {FD:.4f}, s.e.: {American_put_price[3]:.4f}, D.C.: {DC[0]:.4f}, s.e.: {DC[1]:.4f}')

S0/Sigma/T: 90/0.2/1, EUR Put: 10.8414, LSM: 11.9158, Premium: 1.0744, F.D.: 11.7637, s.e.: 0.2711, D.C.: 11.9093, s.e.: 0.6232
S0/Sigma/T: 90/0.2/2, EUR Put: 11.4484, LSM: 13.0711, Premium: 1.6227, F.D.: 13.0708, s.e.: 0.3303, D.C.: 13.2832, s.e.: 0.7600
S0/Sigma/T: 90/0.4/1, EUR Put: 17.9818, LSM: 18.2961, Premium: 0.3143, F.D.: 18.5139, s.e.: 0.5135, D.C.: 18.7789, s.e.: 0.9514
S0/Sigma/T: 90/0.4/2, EUR Put: 21.4763, LSM: 22.4897, Premium: 1.0134, F.D.: 22.4386, s.e.: 0.6211, D.C.: 22.9933, s.e.: 1.1125
S0/Sigma/T: 95/0.2/1, EUR Put: 8.1618, LSM: 8.7700, Premium: 0.6082, F.D.: 8.7437, s.e.: 0.2618, D.C.: 8.9148, s.e.: 0.5853
S0/Sigma/T: 95/0.2/2, EUR Put: 9.2425, LSM: 10.3594, Premium: 1.1169, F.D.: 10.3942, s.e.: 0.3262, D.C.: 10.6283, s.e.: 0.7282
S0/Sigma/T: 95/0.4/1, EUR Put: 15.6903, LSM: 15.8683, Premium: 0.1780, F.D.: 16.1098, s.e.: 0.4803, D.C.: 16.3971, s.e.: 0.9304
S0/Sigma/T: 95/0.4/2, EUR Put: 19.5394, LSM: 20.2437, Premium: 0.7043, F.D.: 20.3707, s.e.: 0.5958, D.C.: 20.

## 2b) Jump Merton <a class="anchor" id="secondb-bullet"></a>

In [30]:
λ = 0.4 # intensity rate of the Poisson process
α = -0.5 # Mean of log-normal jump size
β = 0.4 # Volatility of log-normal jump size

In [31]:
# Store results
Results_JM = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_JM = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Calculate European Put using Jump Merton semi-closed solution
            JumpMerton_price = MC_Option_Pricing_JM.merton_jump_option_value(α, β)
            
            # 2) Compute American Put using LSM
            MC_Option_Pricing_JM.MertonJumpDiffusion_vec(α, β) # simulate jump paths
            American_put_price = MC_Option_Pricing_JM.american_option_LSM(poly_degree, otype='put') # price option
            
            # 3) Calculate premium
            Option_Premium = American_put_price[0] - JumpMerton_price 
            
            # 4) Compute American Put using Dynamic Chebyshev 
            DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, n, n_chebyshev_pol, seed, use_AV)
            # a) offline phase
            domain = DC_option_pricing.calculate_truncated_domain_JumpMerton(α, β)
            xknots = DC_option_pricing.calculate_nodal_points(domain)
            JM = DC_option_pricing.generate_Jump_path(xknots, α, β)
            Γ = DC_option_pricing.compute_generalized_moments(domain,xknots)

            # b) online phase
            DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots,Γ)
            
            # 5) Store the results
            Results_JM.append((S0, σ, T, JumpMerton_price, American_put_price[0], Option_Premium, DC[0]))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, EUR Put: {JumpMerton_price:.4f}, LSM: {American_put_price[0]:.4f}, Premium: {Option_Premium:.4f}, D.C.: {DC[0]:.4f}')

S0/Sigma/T: 90/0.2/1, EUR Put: 15.4293, LSM: 15.2358, Premium: -0.1935, D.C.: 17.6415
S0/Sigma/T: 90/0.2/2, EUR Put: 19.7099, LSM: 20.2854, Premium: 0.5755, D.C.: 23.3843
S0/Sigma/T: 90/0.4/1, EUR Put: 21.3713, LSM: 21.2404, Premium: -0.1309, D.C.: 25.4698
S0/Sigma/T: 90/0.4/2, EUR Put: 26.7443, LSM: 27.4076, Premium: 0.6634, D.C.: 32.6467
S0/Sigma/T: 95/0.2/1, EUR Put: 13.5498, LSM: 13.2739, Premium: -0.2759, D.C.: 15.4845
S0/Sigma/T: 95/0.2/2, EUR Put: 18.1511, LSM: 18.7382, Premium: 0.5870, D.C.: 21.6288
S0/Sigma/T: 95/0.4/1, EUR Put: 19.3825, LSM: 19.2284, Premium: -0.1541, D.C.: 23.5186
S0/Sigma/T: 95/0.4/2, EUR Put: 25.0869, LSM: 25.5603, Premium: 0.4734, D.C.: 31.0005
S0/Sigma/T: 100/0.2/1, EUR Put: 12.0261, LSM: 11.6292, Premium: -0.3969, D.C.: 13.6469
S0/Sigma/T: 100/0.2/2, EUR Put: 16.7778, LSM: 17.1873, Premium: 0.4095, D.C.: 19.9897
S0/Sigma/T: 100/0.4/1, EUR Put: 17.6104, LSM: 17.2227, Premium: -0.3877, D.C.: 21.7062
S0/Sigma/T: 100/0.4/2, EUR Put: 23.5664, LSM: 23.7937, P

## 2c) Constant Elasticity of Variance (CEV) <a class="anchor" id="secondc-bullet"></a>

In [32]:
γ = 1.5 # parameter governing elasticity with respect to price, if set = 2 then we obtain BS prices

In [33]:
# Store results
Results_CEV = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_CEV = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Compute American Put using LSM
            MC_Option_Pricing_CEV.CEV(γ) # simulate CEV paths
            American_put_price = MC_Option_Pricing_CEV.american_option_LSM(poly_degree, otype='put') # price option
            
            # 2) Compute American Put using Dynamic Chebyshev
            DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, n, n_chebyshev_pol, seed, use_AV)
            # a) offline phase
            domain = DC_option_pricing.calculate_truncated_domain_GBM()
            xknots = DC_option_pricing.calculate_nodal_points(domain)
            CEV = DC_option_pricing.generate_CEV_path(xknots, γ)
            Γ = DC_option_pricing.compute_generalized_moments(domain, xknots)

            # b) online phase
            DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots, Γ)
            
            # 3) Store the results
            Results_CEV.append((S0, σ, T, American_put_price[0], DC[0]))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, LSM: {American_put_price[0]:.4f}, D.C.: {DC[0]:.4f}')

S0/Sigma/T: 90/0.2/1, LSM: 9.9255, D.C.: 10.3373
S0/Sigma/T: 90/0.2/2, LSM: 9.8402, D.C.: 10.8464
S0/Sigma/T: 90/0.4/1, LSM: 10.0482, D.C.: 14.5960
S0/Sigma/T: 90/0.4/2, LSM: 10.2632, D.C.: 17.1494
S0/Sigma/T: 95/0.2/1, LSM: 4.9297, D.C.: 6.8071
S0/Sigma/T: 95/0.2/2, LSM: 4.9353, D.C.: 7.7088
S0/Sigma/T: 95/0.4/1, LSM: 6.2868, D.C.: 11.9440
S0/Sigma/T: 95/0.4/2, LSM: 6.9045, D.C.: 14.7843
S0/Sigma/T: 100/0.2/1, LSM: 1.3617, D.C.: 4.2796
S0/Sigma/T: 100/0.2/2, LSM: 1.5167, D.C.: 5.3875
S0/Sigma/T: 100/0.4/1, LSM: 3.6363, D.C.: 9.6798
S0/Sigma/T: 100/0.4/2, LSM: 4.5347, D.C.: 12.7250
S0/Sigma/T: 105/0.2/1, LSM: 0.2798, D.C.: 2.5683
S0/Sigma/T: 105/0.2/2, LSM: 0.4309, D.C.: 3.6999
S0/Sigma/T: 105/0.4/1, LSM: 2.0190, D.C.: 7.8023
S0/Sigma/T: 105/0.4/2, LSM: 2.9312, D.C.: 10.9364
S0/Sigma/T: 110/0.2/1, LSM: 0.0280, D.C.: 1.4772
S0/Sigma/T: 110/0.2/2, LSM: 0.1014, D.C.: 2.5010
S0/Sigma/T: 110/0.4/1, LSM: 0.9367, D.C.: 6.2436
S0/Sigma/T: 110/0.4/2, LSM: 1.7988, D.C.: 9.3831
